In [ ]:
import cv2
from fer import FER
from datetime import datetime
import csv
import os
import time

# CSV log file setup
log_file = "mask_mood_log.csv"
if not os.path.exists(log_file):
    with open(log_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["DateTime", "Emotion", "MaskStatus"])

# Initialize webcam and emotion detector
cam = cv2.VideoCapture(0)
detector = FER(mtcnn=True)

print("Starting real-time detection... Press 'q' to quit.")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Webcam error.")
        break

    # Resize for better speed (optional)
    resized_frame = cv2.resize(frame, (640, 480))

    # Detect emotions
    emotions = detector.detect_emotions(resized_frame)
    if emotions:
        dominant_emotion = max(emotions[0]["emotions"], key=emotions[0]["emotions"].get)
        mask_status = "No Mask"
    else:
        dominant_emotion = "neutral"
        mask_status = "Maybe Mask"

    # Log the result
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_data = [now, dominant_emotion, mask_status]

    with open(log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(log_data)

    # Display frame with label
    label = f"{dominant_emotion} | {mask_status}"
    cv2.putText(resized_frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Real-Time Mood+Mask Logger", resized_frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Stopping...")
        break

    # Optional: delay between logs
    time.sleep(2)  # logs every 2 seconds

cam.release()
cv2.destroyAllWindows()


Starting real-time detection... Press 'q' to quit.
